In [12]:
import pandas as pd
from pathlib import Path

REPORT_FILE_PATH = Path('../llm-answer-results/reports')

In [13]:
pd.read_csv(REPORT_FILE_PATH / '2023-10-18' / 'gpt_llm_auto_score.csv')

,file_name,question,human_answer,llm_answer,score
0,1927.txt,\nKolik ústních jednání ve věci proběhlo?\n,"\nVe věci žádné ústní jednání neproběhlo, účas...","Z textu rozhodnutí není možné zjistit, kolik ú...",2
1,968.txt,\nKdo a kolik celkem musí zaplatit?\n,"\nPlatit musí žalovaný a to celkem 3344,95 Kč.","Žalovaný musí zaplatit žalobkyni celkem 1 855,...",2
2,125.txt,\nO co se v této věci vedl spor? (jednou větou)\n,\nŽalobkyně požadavala zaplacení částky za odb...,V této věci se vedl spor o zaplacení předmětné...,9
3,829.txt,\nOhledně čeho spolu strany uzavřely pojištění?\n,\nŠlo o pojištění odpovědnosti za škodu způsob...,Strany uzavřely pojištění odpovědnosti za škod...,9
4,1912.txt,\nJak často mělo být placeno pojistné?\n,\nPojistné bylo sjednáno s jednou roční splátkou.,Pojistné mělo být placeno jednou ročně.,9
5,666.txt,\nKolik činila přirážka k jízdnému při jízdě n...,\nJednorázová přirážka je 1 500 Kč.,Podle uvedené části rozsudku činila přirážka k...,10
6,474.txt,\nKdo a kolik celkem musí zaplatit?\n,\nPlatit musí žalovaný a to celkem 4535 Kč + ú...,Žalovaný musí zaplatit žalobkyni celkem částku...,8
7,1952.txt,\nJaký typ právního výkladu soud zmiňuje v roz...,\nSoud hovoří o gramatickém a logickém výkladu.,Soud v rozhodnutí zmiňuje gramatický a logický...,8
8,1988_2.txt,\nPodle jakého zákona soud věc posoudil?\n,\nSoud věc posoudil podle zákona č. 89/2012 Sb...,Soud věc posoudil podle § 2395 zákona č. 89/20...,9
9,1988.txt,\nJaký typ smlouvy spolu uzavřeli žalobkyně a ...,\nŽalobkyně a žalovaný spolu uzavřeli smlouvu ...,Žalobkyně a žalovaný uzavřeli smlouvu o úvěru.,10
